# Web Page Summarization Evaluation Workflow

This notebook evaluates the quality of web page summaries generated by different LLM engines using an LLM-as-a-judge approach with G-Eval methodology.

## 1. Creating a Gold Standard

This section creates a gold standard dataset using the GPT-5.2 model to generate high-quality summaries. The gold standard will serve as a reference for evaluating other summarization engines and for distillation tasks.

In [1]:
import json
import asyncio
from agents.summarizer import Summarizer
from tqdm.asyncio import tqdm as atqdm

# Load baseline data
with open('data/baseline_1k.json', 'r', encoding='utf-8') as f:
    baseline_data = json.load(f)

# Extract the data list from the top-level structure
baseline_data = baseline_data['data']
print(f"Loaded {len(baseline_data)} items from baseline dataset")

# Initialize the gold standard summarizer with GPT-5.2
gold_summarizer = Summarizer(model="gpt-5.2-2025-12-11")

# Initialize variables to track total cost and skipped items
total_cost = 0.0
gold_standard_data = []
skipped_items = []

async def summarize_item(item):
    """Async wrapper to summarize a single item"""
    loop = asyncio.get_event_loop()
    # Run the synchronous summarize in a thread pool
    summary, cost = await loop.run_in_executor(
        None, 
        lambda: gold_summarizer.summarize(item['markdown_content'], get_cost=True, allow_long_context=True)
    )
    
    # Check if the request was skipped due to token limits
    if summary is None:
        return None, item['url']
    
    return {
        'url': item['url'],
        'markdown_content': item['markdown_content'],
        'summary': summary
    }, cost

async def generate_summaries():
    """Generate all summaries concurrently"""
    global total_cost, gold_standard_data, skipped_items
    
    # Create tasks for all items (limit concurrency to avoid rate limits)
    max_concurrent = 10  # Adjust based on your API rate limits
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def summarize_with_semaphore(item):
        async with semaphore:
            return await summarize_item(item)
    
    # Process all items concurrently with progress bar
    print("Generating gold standard summaries with GPT-5.2...")
    tasks = [summarize_with_semaphore(item) for item in baseline_data]
    results = await atqdm.gather(*tasks, desc="Processing items")
    
    # Collect results and accumulate costs
    for result in results:
        if result[0] is None:
            # Item was skipped due to token limits
            skipped_items.append(result[1])
        else:
            gold_item, cost = result
            gold_standard_data.append(gold_item)
            total_cost += cost

# Run the async function
await generate_summaries()

# Save the gold standard dataset
output_path = 'data/goldstandard_1k.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(gold_standard_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Gold standard dataset saved to {output_path}")
print(f"✓ Total items processed: {len(gold_standard_data)}")
if skipped_items:
    print(f"⚠ Skipped {len(skipped_items)} items due to token limits:")
    for url in skipped_items:
        print(f"  - {url}")
print(f"✓ Total estimated cost for gold standard generation (based on tokens and rates): ${total_cost:.4f}")

Loaded 1000 items from baseline dataset
Generating gold standard summaries with GPT-5.2...


Processing items:   2%|▏         | 22/1000 [00:22<14:58,  1.09it/s] 

Processing items:  13%|█▎        | 132/1000 [01:58<13:05,  1.10it/s]

Processing items:  18%|█▊        | 183/1000 [02:45<09:00,  1.51it/s]

Processing items:  44%|████▍     | 439/1000 [06:43<08:07,  1.15it/s]

...

Processing items:  52%|█████▏    | 518/1000 [07:57<04:52,  1.65it/s]

Processing items:  65%|██████▍   | 646/1000 [09:54<04:44,  1.24it/s]

Processing items:  79%|███████▉  | 789/1000 [11:57<01:29,  2.35it/s]

Processing items: 100%|██████████| 1000/1000 [15:18<00:00,  1.09it/s]



✓ Gold standard dataset saved to data/goldstandard_1k.json
✓ Total items processed: 992
⚠ Skipped 8 items due to token limits:
  - https://pmc.ncbi.nlm.nih.gov/articles/PMC7271218/
  - https://www.jsog.or.jp/activity/pdf/gl_fujinka_2023.pdf
  - https://weatherspark.com/h/y/557/2024/Historical-Weather-during-2024-in-San-Francisco-California-United-States
  - https://servicehub.ucdavis.edu/servicehub?id=ucd_kb_article&sys_id=cf60ebc293f1e69083cc38797bba1020
  - https://s5.static.brasilescola.uol.com.br/vestibular/2022/12/resultado-cederj-2023.pdf
  - https://colab.research.google.com/github/hc9903/deepke/blob/master/isa.ipynb
  - https://www.insp.mx/resources/images/stories/INSP/Docs/Transparencia/EDICION%202011%20MEDICAMENTOS%20-%20link.pdf
  - https://s2.static.brasilescola.uol.com.br/vestibular/2024/01/resultado-cederj-2024.pdf
✓ Total estimated cost for gold standard generation (based on tokens and rates): $30.9979


For benchmarking, ignoring the long requests is fine. Production will need to handle inference that can handle these requests by cleaning / splitting the requests.

At this point I'll split this gold standard to train-validation subsets, so that all evaluations will be done on the same web-pages.

In [4]:
import json
import tiktoken

PAGE_MAX_TOKENS = 64000

# Load the gold standard dataset
with open('data/goldstandard_1k.json', 'r', encoding='utf-8') as f:
    gold_standard = json.load(f)

print(f"Loaded {len(gold_standard)} items from gold standard")

# Initialize tokenizer for token counting
tokenizer = tiktoken.encoding_for_model("gpt-4o")  # gpt-4.1 uses same tokenizer as gpt-4o
print(f"Using tokenizer: {tokenizer.name}")

# Define constraints for training data
SUMMARY_MAX_CHARS = 1500
PAGE_MAX_TOKENS = 64000

# Identify training candidates: summaries ≤1500 chars AND content ≤64K tokens
training_candidates = []
validation_data = []

for item in gold_standard:
    summary_length = len(item['summary'])
    token_count = len(tokenizer.encode(item['markdown_content']))
    
    # Check if item qualifies for training (both constraints)
    if summary_length <= SUMMARY_MAX_CHARS and token_count <= PAGE_MAX_TOKENS:
        training_candidates.append(item)
    else:
        validation_data.append(item)

print(f"✓ Training candidates: {len(training_candidates)} items (≤{SUMMARY_MAX_CHARS:,} chars & ≤{PAGE_MAX_TOKENS:,} tokens)")
print(f"✓ Validation set base: {len(validation_data)} items (samples not meeting training criteria)")

# Take up to 50% of original dataset for training from candidates
target_train_size = len(gold_standard) // 2
train_data = training_candidates[:target_train_size]

# Add any remaining training candidates to validation
validation_data.extend(training_candidates[target_train_size:])

print(f"\nFinal split:")
print(f"Train set: {len(train_data)} items (selected from candidates)")
print(f"Validation set: {len(validation_data)} items (all remaining samples)")

# Save train set (for distillation)
train_path = 'data/goldstandard_train.json'
with open(train_path, 'w', encoding='utf-8') as f:
    json.dump(train_data, f, indent=2, ensure_ascii=False)

# Save validation set (for evaluation)
validation_path = 'data/goldstandard_validation.json'
with open(validation_path, 'w', encoding='utf-8') as f:
    json.dump(validation_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Train set saved to {train_path}")
print(f"✓ Validation set saved to {validation_path}")
print(f"✓ All training examples meet both constraints: ≤{PAGE_MAX_TOKENS:,} tokens & ≤{SUMMARY_MAX_CHARS:,} chars")

Loaded 992 items from gold standard
Using tokenizer: o200k_base
✓ Training candidates: 404 items (≤1,500 chars & ≤64,000 tokens)
✓ Validation set base: 588 items (samples not meeting training criteria)

Final split:
Train set: 404 items (selected from candidates)
Validation set: 588 items (all remaining samples)

✓ Train set saved to data/goldstandard_train.json
✓ Validation set saved to data/goldstandard_validation.json
✓ All training examples meet both constraints: ≤64,000 tokens & ≤1,500 chars


## 2. Distilling the Gold Standard into a Smaller Model

This part will attempt to distil the intelligence of the GPT-5.2 model into smaller variants of the GPT-4.1 family, in order to create faster-cheaper solutions of similar performance


In [5]:
from train.finetune import prepare_and_train

# Fine-tune both models using the training data
models_to_train = [
    "gpt-4.1-mini-2025-04-14",
    "gpt-4.1-nano-2025-04-14"
]

finetuned_models = {}

for model in models_to_train:
    print(f"\n{'='*70}")
    print(f"Starting fine-tuning for {model}")
    print(f"{'='*70}\n")
    
    finetuned_model_id = prepare_and_train(
        model=model,
        train_json_path='data/goldstandard_train.json',
        n_epochs=3
    )
    
    finetuned_models[model] = finetuned_model_id
    print(f"\n✓ {model} → {finetuned_model_id}\n")

print(f"\n{'='*70}")
print("All fine-tuning jobs completed!")
print(f"{'='*70}")
for base, ft in finetuned_models.items():
    print(f"{base}")
    print(f"  → {ft}")



Starting fine-tuning for gpt-4.1-mini-2025-04-14


Model: gpt-4.1-mini-2025-04-14
Training samples: 404
Total tokens: 2,977,919
Epochs: 3
Estimated training cost: $44.67
Training file: data\train_gpt-4.1-mini-2025-04-14.jsonl

Fine-tuning job submitted: ftjob-GausB92vCt2Mfm7YyAU2V0J2
Monitor at: https://platform.openai.com/finetune/ftjob-GausB92vCt2Mfm7YyAU2V0J2

[1768853681] Created fine-tuning job: ftjob-GausB92vCt2Mfm7YyAU2V0J2
[1768853681] Validating training file: file-Ch4D5iWYsj6r5JYfj1MLVW
[1768853905] Files validated, moving job to queued state
[1768853915] Fine-tuning job started
[1768854076] Step 1/1212: training loss=1.27
[1768854076] Step 2/1212: training loss=1.91
[1768854082] Step 3/1212: training loss=1.41
[1768854084] Step 4/1212: training loss=0.65
[1768854084] Step 5/1212: training loss=1.14
[1768854084] Step 6/1212: training loss=1.41
[1768854088] Step 7/1212: training loss=0.81
[1768854088] Step 8/1212: training loss=1.33
[1768854088] Step 9/1212: training loss=0.9

## Benchmarking

At this point I'll define a small benchmark of 20% of the validation set for model selection, for cost and time efficiency.

In [4]:
import json
import tiktoken

PAGE_MAX_TOKENS = 250000 # Limit for >gpt-5.0 families, to ensure compatibility with the judge

# Load the gold standard validation dataset
with open('data/goldstandard_validation.json', 'r', encoding='utf-8') as f:
    gold_standard = json.load(f)

print(f"Loaded {len(gold_standard)} items from gold standard validation set")

# Initialize tokenizer for token counting (using gpt-4.1 family)
tokenizer = tiktoken.encoding_for_model("gpt-4o")  # gpt-4.1 uses same tokenizer as gpt-4o
print(f"Using tokenizer: {tokenizer.name}")

# Split data based on token count
items_under_limit = []
items_over_limit = []

for item in gold_standard:
    token_count = len(tokenizer.encode(item['markdown_content']))
    if token_count <= PAGE_MAX_TOKENS:
        items_under_limit.append(item)
    else:
        items_over_limit.append(item)

print(f"\nItems under {PAGE_MAX_TOKENS:,} tokens: {len(items_under_limit)}")
print(f"Items over {PAGE_MAX_TOKENS:,} tokens: {len(items_over_limit)}")

# Calculate target benchmarking size (10% of total validation set, ~60 items)
target_size = len(gold_standard) // 10

# Take up to 10% of total for benchmarking (only from items under limit)
validation_data = items_under_limit[:target_size]

print(f"\nBenchmark set: {len(validation_data)} items (all under token limit)")

# Save benchmark set
bm_path = 'data/goldstandard_validation_benchmark.json'
with open(bm_path, 'w', encoding='utf-8') as f:
    json.dump(validation_data, f, indent=2, ensure_ascii=False)

print(f"\n✓ Benchmark set saved to {bm_path}")
print(f"✓ All benchmarking examples are within the {PAGE_MAX_TOKENS:,} token limit for fine-tuning")

# Create baseline subset matching the same URLs
print("\nCreating baseline subset with same URLs...")
with open('data/baseline_1k.json', 'r', encoding='utf-8') as f:
    baseline_full = json.load(f)

# Extract URLs from validation_data
benchmark_urls = {item['url'] for item in validation_data}

# Filter baseline data to match benchmark URLs
baseline_subset = [item for item in baseline_full['data'] if item['url'] in benchmark_urls]

print(f"Matched {len(baseline_subset)} baseline items")

# Save baseline subset
baseline_subset_path = 'data/baseline_validation_benchmark.json'
with open(baseline_subset_path, 'w', encoding='utf-8') as f:
    json.dump(baseline_subset, f, indent=2, ensure_ascii=False)

print(f"✓ Baseline subset saved to {baseline_subset_path}")


Loaded 588 items from gold standard validation set
Using tokenizer: o200k_base

Items under 250,000 tokens: 586
Items over 250,000 tokens: 2

Benchmark set: 58 items (all under token limit)

✓ Benchmark set saved to data/goldstandard_validation_benchmark.json
✓ All benchmarking examples are within the 250,000 token limit for fine-tuning

Creating baseline subset with same URLs...
Matched 58 baseline items
✓ Baseline subset saved to data/baseline_validation_benchmark.json


Benchmark without Retry:

In [2]:
from evaluation.benchmark import BenchmarkingSuite
from agents.config import RATES

# 1. Define models to test (from your RATES keys)
test_models = ['Baseline'] + [model for model in RATES.keys()]

# 2. Prepare subset (validation subset already defined)
subset_data = validation_data

# 3. Run flow
suite = BenchmarkingSuite(rates=RATES)
report_df = suite.run_benchmark(subset_data, test_models)

# 4. Display stylized table
report_df.set_index('model').style.background_gradient(cmap='viridis')


Model 1/10: Baseline
Loading baseline summaries from data/baseline_validation_benchmark.json
✓ Filtered to 99 baseline summaries matching eval subset
✓ Loaded 99 baseline summaries
✓ Saved to data/inference_baseline.json


[1/10] Judging: 100%|██████████| 99/99 [16:07<00:00,  9.78s/sample]



Model 2/10: gpt-4o-mini


[2/10] Inference: 100%|██████████| 99/99 [11:28<00:00,  6.96s/sample]


✓ Saved to data/inference_gpt-4o-mini.json


...

[10/10] Inference: 100%|██████████| 99/99 [13:02<00:00,  7.90s/sample]


✓ Saved to data/inference_gpt-5.2-2025-12-11.json


[10/10] Judging: 100%|██████████| 99/99 [20:05<00:00, 12.18s/sample]



Benchmark Finished. Total Judge Cost: $22.9607


,relevance,faithfulness,coherence,fluency,conciseness,length,latency,est_cost_per_1k
model,,,,,,,,
Baseline,2.282828,4.444444,2.181818,2.838384,2.121212,4.595960,0.000000,0.000000
gpt-4o-mini,4.171717,4.010101,4.909091,5.000000,4.030303,4.797980,5.947505,1.554302
gpt-4.1-2025-04-14,4.515152,4.000000,4.939394,5.000000,4.101010,4.757576,7.144455,20.840869
gpt-4.1-mini-2025-04-14,4.353535,3.969697,4.919192,4.979798,3.989899,4.353535,6.499374,4.196521
gpt-4.1-nano-2025-04-14,4.070707,3.515152,4.676768,4.949495,3.737374,4.595960,2.941758,1.041138
ft:gpt-4.1-mini-2025-04-14:tavily::CzWAcE6p,4.696970,4.494949,4.929293,4.979798,3.949495,2.818182,6.404545,8.798570
ft:gpt-4.1-nano-2025-04-14:tavily::CzX41hjk,4.414141,4.030303,4.797980,4.909091,3.929293,3.020202,3.404394,2.456604
gpt-5-nano,4.474747,4.050505,4.898990,4.909091,4.010101,3.101010,19.274374,1.248511
gpt-5-mini,4.646465,4.393939,4.959596,4.969697,4.131313,4.111111,15.427586,3.941081


Benchmark with Retry:

In [6]:
import importlib
import agents.config
import agents.llm
import agents.summarizer
import evaluation.benchmark

# Reload modules in dependency order
importlib.reload(agents.config)
importlib.reload(agents.llm)
importlib.reload(agents.summarizer)
importlib.reload(evaluation.benchmark)

from evaluation.benchmark import BenchmarkingSuite
from agents.config import RATES

# 1. Define models to test (from your RATES keys)
test_models = [model for model in RATES.keys()] + ['Baseline']

# 2. Prepare subset (validation subset already defined)
subset_data = validation_data

# 3. Run flow
suite = BenchmarkingSuite(rates=RATES)
report_df = suite.run_benchmark(subset_data, test_models)

# 4. Display stylized table
report_df.set_index('model').style.background_gradient(cmap='viridis')


Model 1/12: gpt-4o-mini


[1/12] Inference: 100%|██████████| 58/58 [09:26<00:00,  9.77s/sample]


✓ Saved to data/inference_gpt-4o-mini.json


[1/12] Judging: 100%|██████████| 58/58 [09:54<00:00, 10.26s/sample]


✓ Judge results saved to data/judge_gpt-4o-mini.json

Model 2/12: gpt-4.1-2025-04-14


...

[11/12] Judging: 100%|██████████| 58/58 [09:27<00:00,  9.78s/sample]


✓ Judge results saved to data/judge_gpt-5.2-2025-12-11.json

Model 12/12: Baseline
Loading baseline summaries from data/baseline_validation_benchmark.json
✓ Filtered to 58 baseline summaries matching eval subset
✓ Loaded 58 baseline summaries
✓ Saved to data/inference_baseline.json


[12/12] Judging: 100%|██████████| 58/58 [08:33<00:00,  8.85s/sample]


✓ Judge results saved to data/judge_Baseline.json

Benchmark Finished. Total Judge Cost: $20.1136


,relevance,faithfulness,coherence,fluency,conciseness,length,latency,quality,est_cost_per_1k
model,,,,,,,,,
gpt-4o-mini,4.120690,3.896552,4.862069,5.000000,3.965517,4.949310,8.759172,4.368966,1.235529
gpt-4.1-2025-04-14,4.379310,3.896552,4.913793,4.982759,4.120690,5.000000,7.273259,4.458621,15.243828
gpt-4.1-mini-2025-04-14,4.344828,3.793103,4.931034,4.982759,3.931034,4.880690,9.312966,4.396552,3.628462
gpt-4.1-nano-2025-04-14,4.172414,3.413793,4.586207,4.879310,3.793103,4.940345,4.561276,4.168966,0.812433
ft:gpt-4.1-mini-2025-04-14:tavily::CzWAcE6p,4.534483,4.413793,4.913793,4.965517,3.844828,3.810690,14.750345,4.534483,10.300276
ft:gpt-4.1-nano-2025-04-14:tavily::CzX41hjk,4.379310,3.879310,4.706897,4.862069,3.913793,4.122241,4.899845,4.348276,2.303928
ft:gpt-4.1-mini-2025-04-14:tavily::CzqO8otr,4.637931,4.413793,4.931034,4.965517,4.034483,4.410000,13.615707,4.596552,9.053697
ft:gpt-4.1-nano-2025-04-14:tavily::Czr44ng2,4.293103,3.689655,4.689655,4.827586,3.896552,3.983793,5.545379,4.279310,2.369459
gpt-5-nano,4.344828,3.948276,4.827586,4.913793,4.017241,4.480000,47.240328,4.410345,2.431193
